# Homework 5 RF Accuracy Improvement

This assignment is inspired by examples of Shan-Hung Wu from National Tsing Hua University.

Requirement: improve the accuracy per feature of the following code from 0.03 up to at least 0.40 and accuracy should be more than 0.92

Here are three hints:

    You can improve the ratio by picking out or "creating" several features.
    Tune hyperparameters
    The ratio can be improved from 0.03 up to 0.47.

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns; sns.set()
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel


# load the breast_cancer dataset
init_data = load_breast_cancer()
# these two features have the highest feature_importances_ values
init_data.feature_names = ['mean concave points', 'worst concave points']
(X, y) = load_breast_cancer(return_X_y=True)
print(X.shape)

#DEFAULT (30 Features)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = RandomForestClassifier(n_estimators=75, max_samples=0.8, random_state=1)
model.fit(X, y)
y_pred = model.predict(X)

model.feature_importances_
print(model.feature_importances_)
accuracy = cross_val_score(model, X, y, cv=5).mean()

print('Number of features: ', model.n_features_in_)
print('accuracy:', accuracy)
print('accuracy / number of features', accuracy/model.n_features_in_)
print('\n\n')

best_match = {'accuracy': 0, 
             'accuracy_per_feat': 0,
             'feature': 0,
             'estimator': 0,
             'max_depth': 0,
             'max_samples': 0,
             'random_state': 0}

#Feature Selection (Features 1 to 2 since accuracy/feature must be less than .47)
for feature in range(1,2):
    #Estimator Selection (estimators 1 to 100)
    for estimator in range(1,100):
        for max_depth in range(1,5):
            for random_state in range(1,50):
                for max_samples in range(1, 10):
                    max_samples = float(max_samples)/10.0
                    new_m = SelectFromModel(model, max_features=feature, prefit=True)

                    new_model = RandomForestClassifier(
                        n_estimators=estimator,
                        criterion="entropy",
                        max_depth=max_depth,
                        max_samples=max_samples,
                        random_state=random_state)
                    new_model.fit(X, y)
                    y_pred = new_model.predict(X)

                    new_model.feature_importances_

                    #print(new_model.feature_importances_)

                    accuracy = cross_val_score(new_model, X, y, cv=5).mean()
                    accuracy_per_feat = accuracy/feature

                    #print('\nNumber of features: ', feature)
                    #print('\nNumber of estimators: ', estimator)
                    #print('accuracy:', accuracy)
                    #print('accuracy / number of features:', accuracy_per_feat)

                    if accuracy > best_match['accuracy'] and accuracy_per_feat > 0.45:
                        best_match = {'accuracy': accuracy, 
                         'accuracy_per_feat': accuracy_per_feat,
                         'feature': feature,
                         'estimator': estimator,
                         'max_depth': max_depth,
                         'max_samples': max_samples,
                         'random_state': random_state}

print('\nNumber of features: ', best_match['feature'])
print('Number of estimators: ', best_match['estimator'])
print('accuracy:', best_match['accuracy'])
print('accuracy / number of features', best_match['accuracy_per_feat'])
print('max_depth:', best_match['max_depth'])
print('max_samples:', best_match['max_samples'])
print('random_state:', best_match['random_state'])


# ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
# 'mean smoothness' 'mean compactness' 'mean concavity'
# 'mean concave points' 'mean symmetry' 'mean fractal dimension'
# 'radius error' 'texture error' 'perimeter error' 'area error'
# 'smoothness error' 'compactness error' 'concavity error'
# 'concave points error' 'symmetry error' 'fractal dimension error'
# 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
# 'worst smoothness' 'worst compactness' 'worst concavity'
# 'worst concave points' 'worst symmetry' 'worst fractal dimension']


(569, 30)
Number of features:  30
accuracy: 0.9477310924369748
accuracy / number of features: 0.03159103641456583

Number of features:  2
accuracy: 0.9213438735177867
accuracy / number of features: 0.46067193675889334
